In [1]:
import pandas as pd
import plotly.graph_objects as go
import utils


In [2]:
pair = "USD_JPY"
granularity = "H4"
df_raw = pd.read_pickle(utils.get_his_data_filename(pair, granularity))


In [3]:
non_cols = ['time', 'volume']
mod_cols = [x for x in df_raw.columns if x not in non_cols]
df_raw[mod_cols] = df_raw[mod_cols].apply(pd.to_numeric)

In [4]:
df_raw.columns

Index(['time', 'volume', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_o', 'bid_h',
       'bid_l', 'bid_c', 'ask_o', 'ask_h', 'ask_l', 'ask_c'],
      dtype='object')

In [5]:
df = df_raw[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_c', 'ask_c']].copy()

In [6]:
df.head()

,time,mid_o,mid_h,mid_l,mid_c,bid_c,ask_c
0,2020-01-01T22:00:00.000000000Z,108.651,108.761,108.607,108.673,108.667,108.679
1,2020-01-02T02:00:00.000000000Z,108.671,108.758,108.658,108.742,108.736,108.747
2,2020-01-02T06:00:00.000000000Z,108.744,108.842,108.696,108.839,108.833,108.845
3,2020-01-02T10:00:00.000000000Z,108.836,108.866,108.658,108.694,108.688,108.700
4,2020-01-02T14:00:00.000000000Z,108.697,108.748,108.212,108.529,108.523,108.535


In [7]:
df_raw.columns

Index(['time', 'volume', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_o', 'bid_h',
       'bid_l', 'bid_c', 'ask_o', 'ask_h', 'ask_l', 'ask_c'],
      dtype='object')

In [8]:
df.head()

,time,mid_o,mid_h,mid_l,mid_c,bid_c,ask_c
0,2020-01-01T22:00:00.000000000Z,108.651,108.761,108.607,108.673,108.667,108.679
1,2020-01-02T02:00:00.000000000Z,108.671,108.758,108.658,108.742,108.736,108.747
2,2020-01-02T06:00:00.000000000Z,108.744,108.842,108.696,108.839,108.833,108.845
3,2020-01-02T10:00:00.000000000Z,108.836,108.866,108.658,108.694,108.688,108.700
4,2020-01-02T14:00:00.000000000Z,108.697,108.748,108.212,108.529,108.523,108.535


In [9]:
SLOSS = 0.4
TPROFIT = 0.8
ENTRY_PRC = 0.1

def direction(row):
    if row.mid_c > row.mid_o:
        return 1
    return -1

def get_signal(row):
    if row.mid_h_prev > row.mid_h and row.mid_l_prev < row.mid_l:
        return row.DIRECTION_prev
    return 0

def get_entry_stop(row):
    if row.SIGNAL == 1:
        return (row.RANGE_prev * ENTRY_PRC) + row.mid_h_prev
    elif row.SIGNAL == -1:
        return row.mid_l_prev - (row.RANGE_prev * ENTRY_PRC)
    else:
        return 0
    
def get_stop_loss(row):
    if row.SIGNAL == 1:
        return row.mid_h_prev - (row.RANGE_prev * SLOSS)
    elif row.SIGNAL == -1:
        return row.mid_l_prev + (row.RANGE_prev * SLOSS)
    else:
        return 0
    
def get_take_profit(row):
    if row.SIGNAL == 1:
        return row.mid_h_prev + (row.RANGE_prev * TPROFIT)
    elif row.SIGNAL == -1:
        return row.mid_l_prev - (row.RANGE_prev * TPROFIT)
    else:
        return 0   

In [10]:
df = df_raw[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_c', 'ask_c']].copy()
df['RANGE'] = df.mid_h - df.mid_l
df['mid_h_prev'] = df.mid_h.shift(1)
df['mid_l_prev'] = df.mid_l.shift(1)
df['RANGE_prev'] = df.RANGE.shift(1)
df['DIRECTION'] = df.apply(direction, axis=1)
df['DIRECTION_prev'] = df.DIRECTION.shift(1).fillna(0).astype(int)
df.dropna(inplace=True)
df['SIGNAL'] = df.apply(get_signal, axis=1)
df.reset_index(drop=True, inplace=True)


In [16]:
df['ENTRY'] = df.apply(get_entry_stop, axis=1)
df['STOPLOSS'] = df.apply(get_stop_loss, axis=1)
df['TAKEPROFIT'] = df.apply(get_take_profit, axis=1)

In [18]:
df[df.SIGNAL!=0].head()

,time,mid_o,mid_h,mid_l,mid_c,bid_c,ask_c,RANGE,mid_h_prev,mid_l_prev,RANGE_prev,DIRECTION,DIRECTION_prev,SIGNAL,ENTRY,STOPLOSS,TAKEPROFIT
0,2020-01-02T02:00:00.000000000Z,108.671,108.758,108.658,108.742,108.736,108.747,0.100,108.761,108.607,0.154,1,1,1,108.7764,108.6994,108.8842
4,2020-01-02T18:00:00.000000000Z,108.526,108.572,108.494,108.561,108.511,108.611,0.078,108.748,108.212,0.536,1,-1,-1,108.1584,108.4264,107.7832
12,2020-01-06T02:00:00.000000000Z,107.950,108.104,107.932,108.058,108.052,108.064,0.172,108.122,107.772,0.350,1,1,1,108.1570,107.9820,108.4020
14,2020-01-06T10:00:00.000000000Z,107.988,108.164,107.982,108.120,108.114,108.126,0.182,108.179,107.926,0.253,1,-1,-1,107.9007,108.0272,107.7236
20,2020-01-07T10:00:00.000000000Z,108.466,108.483,108.368,108.438,108.432,108.444,0.115,108.500,108.262,0.238,-1,1,1,108.5238,108.4048,108.6904


In [12]:
df[df.SIGNAL!=0].shape

(804, 14)

In [13]:
df.shape

(4674, 14)

In [14]:
df_plot = df.iloc[200:300]
df_buys = df_plot[df_plot.SIGNAL ==1]
df_sells = df_plot[df_plot.SIGNAL == -1]

In [15]:

fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor='#24A06B',
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color='#2EC886',
    decreasing_line_color='#FF3A4C'
))
fig.add_trace(go.Scatter(
    x=df_buys.time,
    y=df_buys.mid_c,
    mode='markers',
    marker=dict(color="#0ec4ce",size=12)
))
fig.add_trace(go.Scatter(
    x=df_sells.time,
    y=df_sells.mid_c,
    mode='markers',
    marker=dict(color="#043ef9",size=12)
))
fig.update_layout(width=1000,height=400,
    margin=dict(l=10,b=10,t=10),
    font=dict(size=10,color="#e1e1e1"),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False),
    rangebreaks=[
        dict(bounds=["sat", "mon"])
    ]
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True,
)
fig.show()